# MLFlow-ൽ Phi-3 മോഡൽ ഓടിക്കൽ


---------
ഈ നോട്ട്ബുക്ക് Phi-3 mini 4K Instruct എന്ന ഉദാഹരണത്തിൽ ഒരു കസ്റ്റം Python റാപ്പറിന്റെ ഉപയോഗത്തെ വിശദീകരിക്കുന്നു.


In [1]:
# Importing required packages
import mlflow
from mlflow.models import infer_signature
import onnxruntime_genai as og

### Phi-3 Mini 4K മോഡലിനായി ഇഷ്‌ടാനുസൃത പൈത്തൺ ക്ലാസ് നിർവചിക്കൽ


In [2]:
# Defining custom PythonModel class
class Phi3Model(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        # Retrieving model from the artifacts
        model_path = context.artifacts["phi3-mini-onnx"]
        model_options = {
             "max_length": 300,
             "temperature": 0.2,         
        }
    
        # Defining the model
        self.phi3_model = og.Model(model_path)
        self.params = og.GeneratorParams(self.phi3_model)
        self.params.set_search_options(**model_options)
        
        # Defining the tokenizer
        self.tokenizer = og.Tokenizer(self.phi3_model)

    def predict(self, context, model_input):
        # Retrieving prompt from the input
        prompt = model_input["prompt"][0]
        self.params.input_ids = self.tokenizer.encode(prompt)

        # Generating the model's response
        response = self.phi3_model.generate(self.params)

        return self.tokenizer.decode(response[0][len(self.params.input_ids):])

### MLFlow ആർട്ടിഫാക്ട് സൃഷ്ടിക്കുന്നു


In [3]:
# Generating the MLflow model with the custom Python model
input_example = {"prompt": "<|system|>You are a helpful AI assistant.<|end|><|user|>What is the capital of Spain?<|end|><|assistant|>"}
artifact_path = "phi3_mlflow_model"

with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        artifact_path = artifact_path,
        python_model = Phi3Model(),
        artifacts = {
            "phi3-mini-onnx": "cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4",
        },
        input_example = input_example,
        signature = infer_signature(input_example, ["Run"]),
        extra_pip_requirements = ["torch", "onnxruntime_genai", "numpy"],
    )

2024/06/18 14:17:34 INFO mlflow.models.utils: We convert input dictionaries to pandas DataFrames such that each key represents a column, collectively constituting a single row of data. If you would like to save data as multiple rows, please convert your data to a pandas DataFrame before passing to input_example.


### Phi-3നെ MLFlow മോഡലായി ഓടിക്കൽ


In [4]:
# Loading Phi-3 MLFlow model
loaded_model = mlflow.pyfunc.load_model(
    model_uri = model_info.model_uri
)

In [5]:
# Checking MLFlow model's signature
loaded_model.metadata.signature

inputs: 
  ['prompt': string (required)]
outputs: 
  [string (required)]
params: 
  None

In [6]:
# Testing the loaded model
response = loaded_model.predict(
    {"prompt": "<|system|>You are a stand-up comedian.<|end|><|user|>Tell me a joke about atom<|end|><|assistant|>",}
)
print(response)

 Alright, here's a little atom-related joke for you!

Why don't electrons ever play hide and seek with protons?

Because good luck finding them when they're always "sharing" their electrons!

Remember, this is all in good fun, and we're just having a little atomic-level humor!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ഡിസ്ക്ലെയിമർ:
ഈ ഡോക്യുമെന്റ് AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് പരിഭാഷചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിക്കുന്നുവെങ്കിലും, യാന്ത്രിക (ഓട്ടോമേറ്റഡ്) വിവർത്തനങ്ങളിൽ പിശകുകളും തړېകുറവുകളും ഉണ്ടാകാമെന്ന点 ദയവായി ശ്രദ്ധിക്കുക. മൂലഭാഷയിലെ ഒറിജിനൽ ഡോക്യുമെന്റ് പ്രാമാണിക സ്രോതസ്സായി പരിഗണിക്കണം. അത്യാവശ്യ വിവരങ്ങൾക്ക് പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശിപാർശിക്കുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകളുടെയും തെറ്റായ വ്യാഖ്യാനങ്ങളുടെയും ഉത്തരവാദിത്തം ഞങ്ങൾ ഏറ്റെടുക്കുന്നില്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
